### Exist 2021
### Convertir los datasets de las competencias a json y separarlos 75/25 train/test
- Se crea el modelo evomsa pre-entrenado exist2021_Es.evomsa
- En Single f1_score_macro  = 0.7491851693164604
- En Muliple f1_score_macro = 
  - haha2018, mexa3t, misoginia
- La mejora es apenas 0.003 0.01

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [2]:
# Exist2021

df = pd.read_csv("../exist2021_es.csv", usecols=['text', 'task1']) #,   header=0, nrows=50)# , nrows=30, header=1)
df.rename(columns = {'task1':'klass'}, inplace = True)

In [3]:
df.shape

(3541, 2)

In [4]:
df

,text,klass
0,Nadie te va a tratar tan bien como un hombre q...,sexist
1,"@lindagisela74 Que rica putita obediente, afor...",sexist
2,@BicireporteraDF Yo lo hice a los 18 años por ...,non-sexist
3,las cosas q sueño son indicios de que yo enrea...,non-sexist
4,"Pero a la niña le gustó desde que lo vió, así ...",non-sexist
...,...,...
3536,"Estamos igual sin pareja, pero puedes besar a ...",non-sexist
3537,2020 hijo de re mil putas,non-sexist
3538,SEGURAMENTE ESTA CHICA NO COBRA EL DINERO QUE ...,non-sexist
3539,@safetyaitana mi madre dice q va fea y i agree,sexist


In [5]:
np.unique(df['klass'])

array(['non-sexist', 'sexist'], dtype=object)

In [6]:
df["klass"] = np.where(df["klass"] == "sexist", 1, 0)
# sexist = 1
# No sexist = 

In [7]:
df

,text,klass
0,Nadie te va a tratar tan bien como un hombre q...,1
1,"@lindagisela74 Que rica putita obediente, afor...",1
2,@BicireporteraDF Yo lo hice a los 18 años por ...,0
3,las cosas q sueño son indicios de que yo enrea...,0
4,"Pero a la niña le gustó desde que lo vió, así ...",0
...,...,...
3536,"Estamos igual sin pareja, pero puedes besar a ...",0
3537,2020 hijo de re mil putas,0
3538,SEGURAMENTE ESTA CHICA NO COBRA EL DINERO QUE ...,0
3539,@safetyaitana mi madre dice q va fea y i agree,1


In [8]:
X_train, X_test, y_train, y_test = train_test_split(df['text'], df['klass'], train_size=0.75)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((2655,), (886,), (2655,), (886,))

In [9]:
type(X_train)

pandas.core.series.Series

In [10]:
df_train = pd.concat([X_train, y_train], axis=1)
print(df_train.shape)
df_train

(2655, 2)


,text,klass
2674,@PlaysOfTheWeek Jugada futbolera si no va agua...,1
1656,@SrtaTumba Y que eres más de sodomizar al LoL.,0
2024,@ElEndika @MaxLovesDragas @silnersessian @Lady...,1
3053,@AteoAnalitico @pambish Y lo mas complicado de...,1
2409,"@ElSrDerecha Sr. Fascista, ¿Igualdad real entr...",1
...,...,...
831,igual prefiero q vayan con james a q con el gr...,0
3250,"La imagen del ""brujo"" es escasa, incluso en in...",0
801,"A todo señor, todo honor.... Me acaban de llam...",0
2752,"Si es que Máximo Pradera,lo del empoderamiento...",0


In [11]:
df_test = pd.concat([X_test, y_test], axis=1)
print(df_test.shape)
df_test

(886, 2)


,text,klass
996,@SchrodingerGata @YouTubeEspanol Existe el lib...,0
138,Grandísimos hijos de puta. Seguro que ni multa...,0
719,@minervaramoss1 @Kevinn8_ @carlosbntz5 Lávate ...,1
1811,@Lmarangonib @TattyCanPo @CynthiaViteri6 Mujer...,1
426,@AlvArrCarlos Que no podamos entender a las mu...,1
...,...,...
2927,@Icarte_Ro Y entre el minuto 75 y 90 gol de la...,0
1111,No terminabamos de dejar atrás el tiempo en qu...,1
1423,@Chica_MLC @lilyht_lilyht27 @JBALVIN No le hag...,1
2403,"@aburrido354 A TODA MECHA, A TODA MECHATU NO T...",1


In [12]:
with open('..\exist2021_Es_train.json', 'w') as f:
    f.write(df_train.to_json(orient='records', lines=True))
    
with open('..\exist2021_Es_test.json', 'w') as f:
    f.write(df_test.to_json(orient='records', lines=True))

In [13]:
from EvoMSA.base import EvoMSA
from microtc.utils import save_model
from sklearn import metrics
from microtc.utils import load_model
from microtc.utils import save_model

In [14]:
evo = EvoMSA(TR=True, B4MSA=False, lang='es', Emo=True, Aggress=True, models=None)

# Pq a .squeeze() ? 

In [15]:
evo.fit(X_train.squeeze(), y_train.squeeze())
pred = evo.predict(X_test.squeeze())
print(metrics.f1_score(y_test, pred, average="macro"))

100%|██████████| 30/30 [00:26<00:00,  1.12it/s]

0.7491851693164604


In [16]:
# crear modelo pre-entrenado en Sigle par evitar efecto cascada
save_model(evo, '..\\..\\single_models\\exist21_single_Es.evomsa') 

### Multiple
----------------------------------

In [ ]:
stacked_method = "sklearn.svm.LinearSVC"
# usar = "sklearn.naive_bayes.BernoulliNB"
# usar = "sklearn.naive_bayes.MultinomialNB"

reverse_model = [load_model('..\\..\\models\\text_model_inv_simple.evomsa'), stacked_method]
haha_model = [load_model('..\\..\\models\\haha2018_Es.evomsa'), stacked_method]
mexa3t_model = [load_model('..\\..\\models\\mexa3t2018_aggress_Es.evomsa'), stacked_method]
misoginia_model = [load_model('..\\..\\models\\misoginia_Es.evomsa'), stacked_method]

mm = [haha_model, mexa3t_model, misoginia_model]

evo = EvoMSA(TR=True, B4MSA=False, lang='es', Emo=True, Aggress=True, stacked_method=stacked_method, models= mm)

In [ ]:
evo.fit(X_train.squeeze(), y_train.squeeze())
pred = evo.predict(X_test.squeeze())
print(metrics.f1_score(y_test, pred, average="macro"))

In [ ]:
# crear modelo pre-entrenado
save_model(evo, '..\\..\\models\\exist2021_Es.evomsa') 

In [ ]:
# probarlo
_ = load_model('..\\..\\models\\exist2021_Es.evomsa') 
pred = _.predict(X_test.squeeze())
print(metrics.f1_score(y_test, pred, average="macro"))